In [1]:
import numpy as np
import numpy.linalg
import pandas as pd
from dataclasses import dataclass, field

In [2]:
@dataclass
class SLAE:
    """Class for Systems of Algebraic Linear Equations"""
    name: str
    a: np.array = field(default=None, repr=False)
    b: np.array = field(default=None, repr=False)
    dim: tuple[int] = field(default=None, repr=False)
    dir_name: str = field(default='SLAE', repr=False)
    csv_names: tuple[str] = field(default=('A', 'b', 'x'), repr=False)

    @property
    def get_a(self) -> np.array:
        if self.a is None:
            self.a = pd.read_excel(f'{self.dir_name}/{self.name}.xlsx',
                                   sheet_name=self.csv_names[0],
                                   header=None).to_numpy()
        return self.a

    @property
    def get_dim(self) -> tuple[int]:
        if self.dim is None:
            self.dim = self.get_a.shape
        return self.dim

    @property
    def get_b(self) -> np.array:
        if self.b is None:
            return pd.read_excel(f'{self.dir_name}/{self.name}.xlsx',
                                 sheet_name=self.csv_names[1],
                                 header=None).to_numpy()
        return self.b

    def set_b(self, b, new_name_b_x=()):
        na, ma = self.get_dim
        nb, = b.shape
        if nb != na:
            return False
        self.b = b
        if len(new_name_b_x) != 0 :
            self.csv_names = new_name_b_x
        self.SLAE_to_xlsx()
        df = pd.DataFrame(b, index=None, columns=None)
        a_name, b_name, x_name = self.csv_names
        df.to_csv(f'{self.dir_name}/CSV/{self.name}_{b_name}.csv',
                  header=None, index=None)
        self.SLAE_to_csv()
        return True

    def set_b_zero(self):
        n, m = self.get_a.shape
        return self.set_b(np.zeros(n))

    def x(self, solver=np.linalg.solve) -> np.array:
        a_matr = self.get_a
        n, m = self.get_a.shape
        if n == m and np.linalg.det(a_matr) != 0:
            return solver(a_matr, self.get_b)
        return None

    def SLAE_to_csv(self, index_csv=None, columns_csv=None) -> None:
        dir_csv = f'{self.dir_name}/CSV'
        a_name, b_name, x_name = self.csv_names
        for item, name in zip((self.get_a, self.get_b, self.x()),
                                         (a_name, b_name, x_name)):
            df = pd.DataFrame(item, index=index_csv, columns=columns_csv)
            df.to_csv(f'{dir_csv}/{self.name}_{name}.csv',
                      header=columns_csv is not None,
                      index=index_csv is not None)

    def SLAE_to_xlsx(self, index_csv=None, columns_csv=None) -> None:
        with pd.ExcelWriter(f'{self.dir_name}/{self.name}.xlsx') as writer:
            for item, name in zip((self.a, self.b, self.x()), self.csv_names):
                df = pd.DataFrame(item, index=index_csv, columns=columns_csv)
                df.to_excel(writer, sheet_name=name,
                            header=columns_csv is not None,
                            index=index_csv is not None)


@dataclass
class SLAEhomogeneous(SLAE):
    """Class for Systems of Homogeneous Algebraic Linear Equations"""
    csv_names: tuple[str] = ('A0', 'charact', 'x')

    @property
    def get_b(self) -> np.array:
        dim = self.get_a.shape
        n, m = dim
        self.b = np.zeros(n)
        self.dim = dim
        return self.b

    def set_b(self, b, new_name_b_x=()):
        print('b = 0 in homogeneous SLAE, use get_b instead')


    def x(self) -> np.array:
        return np.zeros(self.get_a.shape[0])

    def SLAE_to_xlsx(self, index_csv=None, columns_csv=None) -> None:
        index = index_csv is not None
        columns = columns_csv is not None
        a_name, charact_name, x_name = self.csv_names
        a_matr = self.get_a
        with pd.ExcelWriter(f'{self.dir_name}/{self.name}.xlsx') as writer:
            for item, name in zip((a_matr, self.x()), (a_name, x_name)):
                df = pd.DataFrame(item, index=index_csv, columns=columns_csv)
                df.to_excel(writer, sheet_name=name,
                            index=index, header=columns)
            a_rank = np.linalg.matrix_rank(a_matr)
            n, m = self.get_dim
            index_charact = ('rank', 'n eq',
                                              'n var', 'null dim')
            df_charact = pd.DataFrame((a_rank, n, m,
                                       m - a_rank),
                                       index=index_charact)
            df_charact.to_excel(writer, sheet_name=charact_name, header=False)



    def SLAE_to_csv(self, index_csv=None, columns_csv=None) -> None:
        dir_csv = f'{self.dir_name}/CSV'
        a_name, charact_name, x_name = self.csv_names
        a_matr = self.get_a
        if self.a is None:
            self.a = self.get_a
        for item, name in zip((self.a, self.x()),
                              (a_name, x_name)):
            df = pd.DataFrame(item, index=index_csv, columns=columns_csv)
            df.to_csv(f'{dir_csv}/{self.name}_{name}.csv',
                      header=columns_csv is not None,
                      index=index_csv is not None)
        a_rank = np.linalg.matrix_rank(a_matr)
        n, m = self.get_dim
        index_charact = ('rank', 'n eq',
                                              'n var', 'null dim')
        df_charact = pd.DataFrame((a_rank, n, m,
                                       m - a_rank),
                                       index=index_charact)
        df_charact.to_csv(f'{dir_csv}/{self.name}_{charact_name}.csv',
                      header=False)





@dataclass
class SLAEnonsquare(SLAE):
    """Class for Systems of Homogeneous Algebraic Linear Equations"""
    pinv: np.array = field(default=None, repr=False)

    def get_pinv(self, if_print=False):
        if if_print:
            print('pinv(self)')
        pinv_matr = np.linalg.pinv(self.get_a)
        self.pinv = pinv_matr
        return pinv_matr

    def x(self) -> np.array:
        a_matr = self.get_a
        n, m = self.get_dim
        if n == m and np.linalg.det(a_matr) != 0:
            return SLAE.x(self, a_matr, self.get_b)
        return np.linalg.pinv(a_matr) @ self.get_b

In [5]:
SLAE_1 = SLAE(name='ab1', dir_name='ab_xlsx')
A = SLAE_1.get_a()

TypeError: 'numpy.ndarray' object is not callable

In [4]:
print(f'SLAE_1.a {SLAE_1.a}')
print(f'SLAE_1.get_a\n {np.round(SLAE_1.get_a, 1)}')
print(f'SLAE_1.a\n {np.round(SLAE_1.a, 1)}')
SLAE_1.set_b_zero()
print(f'SLAE_1.b {SLAE_1.b}')
SLAE_1.SLAE_to_csv()
SLAE_1.set_b(np.array([1, 2, 3, 4, 5, 6, 7]))
print(f'SLAE_1.b {SLAE_1.b}')

SLAE_1.a None


FileNotFoundError: [Errno 2] No such file or directory: 'SLAE/SLAE_1_1.xlsx'

In [7]:
SLAE_1.set_b(np.array([-1, 2, -3, 4, -5, 6, 7]))
print(SLAE_1.b, SLAE_1.x().round(2), sep='\n')

[-1  2 -3  4 -5  6  7]
[-1.16 -0.05  0.57 -0.88  0.57  1.14 -0.41]


In [9]:
SLAEhomo_1 = SLAEhomogeneous(name='SLAE_homo_1')
SLAEhomo_1.SLAE_to_csv()
print(f'SLAE_1.b {SLAEhomo_1.b}')
print(f'SLAE_1.get_b {SLAEhomo_1.get_b}')
print(f'SLAE_1.b {SLAEhomo_1.b}')
SLAEhomo_1.set_b(np.array([1, 2, 3, 4, 5, 6, 7]))
print(f'SLAE_1.b {SLAEhomo_1.b}')
print(f'SLAE_1.a[:2, :5]\n{SLAEhomo_1.a[:2, :5]}')
SLAEhomo_1.SLAE_to_xlsx()

SLAE_1.b None
SLAE_1.get_b [0. 0. 0. 0. 0. 0. 0.]
SLAE_1.b [0. 0. 0. 0. 0. 0. 0.]
b = 0 in homogeneous SLAE, use get_b instead
SLAE_1.b [0. 0. 0. 0. 0. 0. 0.]
SLAE_1.a[:2, :5]
[[8.79597859 4.87523014 8.68212102 4.00944011 9.80170238]
 [0.53146707 9.12385915 7.3379975  6.74437741 7.36291863]]


In [10]:
SLAEnonsq_1 = SLAEnonsquare(name='SLAE_nonsq_1')
a1 = SLAEnonsq_1.get_a
print(f'SLAE_1.get_a[:2, :2]\n{a1[:2, :5]}')
SLAEnonsq_1.set_b_zero()
print(f'SLAE_1.b\n{SLAEnonsq_1.b}')
#SLAEnonsq_1.SLAE_to_csv()
print(f'SLAE_1.b {SLAEnonsq_1.b}')
print(f'SLAE_1.x() {SLAEnonsq_1.x()}')
SLAEnonsq_1.set_b(np.array([1, 2, 3, 4, 5]))
print(f'SLAE_1.get_b {SLAEnonsq_1.get_b}')
x1 = SLAEnonsq_1.x()
a1pinv = SLAEnonsq_1.pinv
print(f'SLAE_1.x() {np.round(x1, 3)}')
SLAEnonsq_1.get_pinv()
print(f'Ax = {a1 @ x1}\nA+ = {a1pinv},\nSLAEnonsq_1.pinv\n{SLAEnonsq_1.pinv}')

SLAE_1.get_a[:2, :2]
[[8.79597859 4.87523014 8.68212102 4.00944011 9.80170238]
 [0.53146707 9.12385915 7.3379975  6.74437741 7.36291863]]
SLAE_1.b
[0. 0. 0. 0. 0.]
SLAE_1.b [0. 0. 0. 0. 0.]
SLAE_1.x() [0. 0. 0. 0. 0. 0. 0.]
SLAE_1.get_b [1 2 3 4 5]
SLAE_1.x() [ 0.265  0.574  0.075 -0.006 -0.552  0.299 -0.107]
Ax = [1. 2. 3. 4. 5.]
A+ = None,
SLAEnonsq_1.pinv
[[ 0.01001078 -0.08763746  0.08232778  0.01573919  0.02403768]
 [-0.13126998  0.03714209  0.14700085  0.03219609  0.01223957]
 [ 0.00443973  0.02346419 -0.04266426 -0.011726    0.03964946]
 [-0.00905401  0.0599786  -0.08256147  0.02450091  0.00653178]
 [ 0.14071037 -0.01108836  0.00177316 -0.01023757 -0.12706106]
 [-0.03522309 -0.00129122 -0.08133019  0.11953186  0.02045715]
 [ 0.03968242  0.02847418 -0.0336971  -0.09250257  0.05352678]]


In [11]:
SLAEnonsq_2_1 = SLAEnonsquare(name='SLAE_nonsq_2')
for _ in range(3):
    SLAEnonsq_2_1.get_pinv(if_print=True)

pinv(self)
pinv(self)
pinv(self)


In [12]:
for _ in range(3):
    print(np.round(SLAEnonsq_2_1.pinv[:2, :3], 2))

[[ 0.01 -0.05  0.03]
 [-0.12  0.02  0.15]]
[[ 0.01 -0.05  0.03]
 [-0.12  0.02  0.15]]
[[ 0.01 -0.05  0.03]
 [-0.12  0.02  0.15]]


In [13]:
SLAEnonsq_2_1 = SLAEnonsquare(name='SLAE_nonsq_2')
for k in range(2):
    print(f'k= {k}, SLAEnonsq_2_1.pinv:\n{SLAEnonsq_2_1.pinv}')
    SLAEnonsq_2_1.get_pinv(if_print=True)

k= 0, SLAEnonsq_2_1.pinv:
None
pinv(self)
k= 1, SLAEnonsq_2_1.pinv:
[[ 0.00727264 -0.04936357  0.02868651  0.05520022  0.07822979 -0.09219359
  -0.01435143]
 [-0.12495814  0.01754926  0.14961972 -0.01106575  0.0262947   0.00570959
   0.04318068]
 [ 0.02406972  0.0149601  -0.0308991  -0.05377123 -0.00857279  0.15435297
  -0.03884185]
 [-0.01732299  0.12546674 -0.19258654  0.10011572  0.11861312 -0.18091091
  -0.00953903]
 [ 0.13320502 -0.03876464 -0.000945   -0.01294965 -0.12988062  0.02819565
   0.03639909]]
pinv(self)


In [14]:
SLAEnonsq_2 = SLAEnonsquare(name='SLAE_nonsq_2')
a2 = SLAEnonsq_2.get_a
print(f'SLAEnonsq_2.a().shape {a2.shape}')
SLAEnonsq_2.set_b_zero()
print(f'SLAEnonsq_2.b {SLAEnonsq_2.b}')
SLAEnonsq_2.SLAE_to_csv()
print(f'SLAEnonsq_2.get_b {SLAEnonsq_2.get_b}')
print(f'SLAEnonsq_2.x() {SLAEnonsq_2.x()}')
SLAEnonsq_2.set_b(np.array([1, 2, 3, 4, 5, -1, -3]))
b2 = SLAEnonsq_2.get_b
print(f'SLAEnonsq_2.get_b {b2}')
x2 = SLAEnonsq_2.x()
a2pinv = SLAEnonsq_2.get_pinv()
print(f'SLAEnonsq_2.x() {x2}')
print(f'A+Ax = {a2pinv @ a2 @ x2}\nA+b = {a2pinv @ b2}')
SLAEnonsq_2.SLAE_to_xlsx()

SLAEnonsq_2.a().shape (7, 5)
SLAEnonsq_2.b [0. 0. 0. 0. 0. 0. 0.]
SLAEnonsq_2.get_b [0. 0. 0. 0. 0. 0. 0.]
SLAEnonsq_2.x() [0. 0. 0. 0. 0.]
SLAEnonsq_2.get_b [ 1  2  3  4  5 -1 -3]
SLAEnonsq_2.x() [ 0.74180272  0.3109584  -0.33448366  0.8589074  -0.78575384]
A+Ax = [ 0.74180272  0.3109584  -0.33448366  0.8589074  -0.78575384]
A+b = [ 0.74180272  0.3109584  -0.33448366  0.8589074  -0.78575384]
